In [12]:
# %%
import pandas as pd
import os
import sys
from Bio.SubsMat import MatrixInfo as matlist
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

def fasta2List(pathFasta):
    f = open(pathFasta, "r")
    title = []
    seq = []
    seq_temp = []
    for line in f:
        if line[0] == ">":
            seq.append(''.join(seq_temp).replace("\n", ""))
            title.append(line.replace("\n", ""))
            seq_temp = []
        else:
            seq_temp.append(line)
    seq.append(''.join(seq_temp).replace("\n", ""))
    seq.pop(0)
    dictionary = dict(zip(title, seq))
    return dictionary

error_list = pd.read_csv("../../data/mismatch-analysis/uniprot-error-mismatch/uniprot_new_errors_filt.txt", sep=" ", header=None)
    

In [127]:
# Flagging des mismatch conservés
counter = 0
for index, row in error_list.iloc[:,:].iterrows():

    conserved = 0

    mySeq = fasta2List("../../data/raw/uniprot-sequence/"+row[0][19:-6])
    protein_Mismatch = [val for key, val in mySeq.items() if row[2] in key]

    mismtaching_seq = protein_Mismatch[0][row[3]:row[4]]
    for i, j in mySeq.items():
        myAlign = pairwise2.align.localms(j, mismtaching_seq, 2, 0, -.5, -.1, one_alignment_only=True, score_only=True)
        #print(format_alignment(*myAlign[0]))
        if myAlign/len(mismtaching_seq) > 1.8:
            conserved +=1

    if conserved >= 4:
        print(str(index)+'\t'.join([str(x) for x in row]))
        counter +=1
print(counter)


2214/home/meyer/uniprot/Q03721.id.fasta.mafft	SEQ_ERROR3	H2N6G1	603	623	602	631
12382/home/meyer/uniprot/Q05C16.id.fasta.mafft	SEQ_ERROR3	G1QQR1	512	587	511	576
12386/home/meyer/uniprot/Q05C16.id.fasta.mafft	SEQ_ERROR3	F6YTR7	511	587	510	576
12387/home/meyer/uniprot/Q05C16.id.fasta.mafft	SEQ_ERROR3	A0A096MTY2	511	587	510	576
12388/home/meyer/uniprot/Q05C16.id.fasta.mafft	SEQ_ERROR3	G3QPK1	512	587	511	576
12450/home/meyer/uniprot/Q07283.id.fasta.mafft	SEQ_ERROR3	A0A0D9S5Q7	870	881	1349	1366
12452/home/meyer/uniprot/Q07283.id.fasta.mafft	SEQ_ERROR3	A0A0D9S5Q7	1064	1075	1596	1608
12528/home/meyer/uniprot/Q08174.id.fasta.mafft	SEQ_ERROR3	A0A0D9RJ19	1043	1246	1029	1055
12529/home/meyer/uniprot/Q08174.id.fasta.mafft	SEQ_ERROR3	G1REY1	1012	1215	1029	1055
12530/home/meyer/uniprot/Q08174.id.fasta.mafft	SEQ_ERROR3	F6QZS4	1038	1241	1029	1055
12532/home/meyer/uniprot/Q08174.id.fasta.mafft	SEQ_ERROR3	F7E1W0	1043	1116	1029	1053
12533/home/meyer/uniprot/Q08174.id.fasta.mafft	SEQ_ERROR3	A0A096NY96	103